In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
%matplotlib inline
plt.style.use('seaborn-v0_8-dark-palette')

In [2]:
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score

In [3]:
from catboost import CatBoostClassifier 

In [4]:
df = pd.read_csv('data.csv')
df.shape

(7043, 21)

In [5]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [7]:
df.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [8]:
df.duplicated().sum()

0

In [9]:
profile = ProfileReport(df, title="Profiling Report", explorative=True, minimal=True)
#profile.to_file('report.html')
#profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
df[df['PhoneService'] == 'No']['MultipleLines'].value_counts()

MultipleLines
No phone service    682
Name: count, dtype: int64

In [11]:
df[df['InternetService'] == 'No'][['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']].value_counts()

OnlineSecurity       OnlineBackup         DeviceProtection     TechSupport          StreamingTV          StreamingMovies    
No internet service  No internet service  No internet service  No internet service  No internet service  No internet service    1526
Name: count, dtype: int64

По результатам анализа данных видим, что 
1) Пропусков нет, заполнять ничего не нужно
2) Дубликатов нет, удалять записи не нужно
3) Данные в целом сбалансированы, нет значительногого превышение одного класса над другим. Единственный несблансированный PhoneService, в нем данные - 90/10.  
План действий:
1) Выкинем признак PhoneService, так как его дублирует MultipleLines, только в первом 2 категории а во втором 3, что более показательно, применим к нему OneHotEncoding
2) Признак InternetService дублирует данные из OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies. Предлагается оставить InternetService и применить OneHot, для остальных признаков вместо трех категорий оставить 1(чтобы не дублироваь много раз 'No internet service' и 'No')
3) В таких полей как Partner, Dependents,PaperlessBilling, Churn,  заменим значения Yes и No на 1 и 0
4) TotalCharges типа object, необходимо привести к числовому
5) Проверить корреляцию и распределние числовых данных


In [12]:
df.drop(columns=['PhoneService'], inplace=True)

In [13]:
obj_to_num_features = ['TotalCharges']
yes_no_features = ['Partner', 'Dependents', 'PaperlessBilling']
internet_services = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
num_features = ['tenure', 'MonthlyCharges']

In [14]:
def change_yes_to_1(X: pd.DataFrame, y=None)->pd.DataFrame:
    for c in X.columns:
        X[c] = X.loc[:,c].map({'Yes': 1, 'No': 0})
    return X

In [15]:
def fill_not_numeric(X: pd.DataFrame, y=None)->pd.DataFrame:
    for c in X.columns:
        X[c] = X[c].apply(pd.to_numeric, errors="coerce")
        X[c] = X[c].fillna(X[X[c].notna()][c].mean())

    return X

In [16]:
def change_yes_to_1_for_InternetServices(X: pd.DataFrame, y=None)->pd.DataFrame:
    for c in X.columns:
        X[c] = X.loc[:,c].map({'Yes': 1, 'No': 0, 'No internet service': 0})
    return X

In [17]:
one_hot_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown="ignore"))])

obj_to_num_transformer = Pipeline(steps=[
    ('fill_not_numeric', FunctionTransformer(fill_not_numeric)),
    ('scaler', StandardScaler())
    ])

change_yes_to_1_transformer = Pipeline(steps=[
    ('change_yes_to_1', FunctionTransformer(change_yes_to_1))])

change_yes_to_1_for_InternetServices_transformer = Pipeline(steps=[
    ('change_yes_to_1_for_InternetServices', FunctionTransformer(change_yes_to_1_for_InternetServices))])

num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

data_transformer = ColumnTransformer(transformers=[
    ('obj_to_num_features', obj_to_num_transformer, obj_to_num_features),
    ('yes_no_features', change_yes_to_1_transformer, yes_no_features),
    ('MultipleLines_feature', one_hot_transformer, ['MultipleLines']),
    ('InternetService_feature', one_hot_transformer, ['MultipleLines']),
    ('InternetServices_feature', change_yes_to_1_for_InternetServices_transformer, internet_services),
    ('num_features', num_transformer, num_features)
])

preprocessor = Pipeline(steps=[("data_transformer", data_transformer)])

In [18]:
preprocessor

Pipeline(steps=[('data_transformer',
                 ColumnTransformer(transformers=[('obj_to_num_features',
                                                  Pipeline(steps=[('fill_not_numeric',
                                                                   FunctionTransformer(func=<function fill_not_numeric at 0x7f1459685da0>)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['TotalCharges']),
                                                 ('yes_no_features',
                                                  Pipeline(steps=[('change_yes_to_1',
                                                                   FunctionTransformer(func=<function change_yes_to_1 at...
                                                 ('InternetServices_feature',
                                                  Pipeline(steps=[('change_yes_to_1_for_InternetServices',
                                                                   FunctionTransformer(func=<function change_yes_to_1_for_InternetServices at 0x7f1459687ec0>))]),
                                                  ['OnlineSecurity',
                                                   'OnlineBackup',
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies']),
                                                 ('num_features',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['tenure',
                                                   'MonthlyCharges'])]))])

In [19]:
preprocessor.fit_transform(df).shape

(7043, 18)

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop(columns='Churn'), df['Churn'], test_size=0.8, random_state=241)

In [24]:
classifier_pipline = Pipeline(
    steps=[("preprocessor", preprocessor), 
           ("classifier", CatBoostClassifier())])

In [25]:
classifier_pipline.fit(X_train, Y_train)

Learning rate set to 0.011923
0:	learn: 0.6865042	total: 1.9ms	remaining: 1.89s
1:	learn: 0.6815911	total: 2.75ms	remaining: 1.37s
2:	learn: 0.6737465	total: 4.34ms	remaining: 1.44s
3:	learn: 0.6664509	total: 6.54ms	remaining: 1.63s
4:	learn: 0.6597100	total: 9.17ms	remaining: 1.82s
5:	learn: 0.6535548	total: 11.6ms	remaining: 1.93s
6:	learn: 0.6474949	total: 13.8ms	remaining: 1.95s
7:	learn: 0.6416458	total: 15.6ms	remaining: 1.93s
8:	learn: 0.6344087	total: 17ms	remaining: 1.87s
9:	learn: 0.6287861	total: 18.5ms	remaining: 1.84s
10:	learn: 0.6238852	total: 20.6ms	remaining: 1.85s
11:	learn: 0.6189644	total: 22.1ms	remaining: 1.82s
12:	learn: 0.6133405	total: 23.5ms	remaining: 1.79s
13:	learn: 0.6083573	total: 24.8ms	remaining: 1.75s
14:	learn: 0.6038755	total: 26.1ms	remaining: 1.72s
15:	learn: 0.5990496	total: 27.8ms	remaining: 1.71s
16:	learn: 0.5947126	total: 29.1ms	remaining: 1.68s
17:	learn: 0.5897408	total: 30.4ms	remaining: 1.66s
18:	learn: 0.5855145	total: 31.7ms	remaining: 1

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('data_transformer',
                                  ColumnTransformer(transformers=[('obj_to_num_features',
                                                                   Pipeline(steps=[('fill_not_numeric',
                                                                                    FunctionTransformer(func=<function fill_not_numeric at 0x7f1459685da0>)),
                                                                                   ('scaler',
                                                                                    StandardScaler())]),
                                                                   ['TotalCharges']),
                                                                  ('yes_no_features',
                                                                   Pipeline(steps=[('change_yes_to_1',
                                                                                    FunctionTransformer...
                                                                                    FunctionTransformer(func=<function change_yes_to_1_for_InternetServices at 0x7f1459687ec0>))]),
                                                                   ['OnlineSecurity',
                                                                    'OnlineBackup',
                                                                    'DeviceProtection',
                                                                    'TechSupport',
                                                                    'StreamingTV',
                                                                    'StreamingMovies']),
                                                                  ('num_features',
                                                                   Pipeline(steps=[('scaler',
                                                                                    StandardScaler())]),
                                                                   ['tenure',
                                                                    'MonthlyCharges'])]))])),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x7f141b9476d0>)])

In [31]:
roc_auc_score(Y_train, classifier_pipline.predict_proba(X_train)[:,1])

0.969669698117974

In [32]:
roc_auc_score(Y_test, classifier_pipline.predict_proba(X_test)[:,1])

0.8172701165606171